# DPR Automation ( Additional Checks for mischivious data from sites)

### What's New (key features)- last update on 27-10-2023: 
1. It reports the charged values. 
2. It checks for the negative value. 
3. This version reverts back the stage in case of failure of code run
4. It saves Data Bank stages for the last 7 days. 
5. This version now includes the vendor that is common in both Steel and MDPE sheet.
6. Bug fixed in "drop duplicates for final_df" --> Line 318
7. Automatically send the Data Bank and site DPR for reports (progress of 15th and last day of month)
8. Sends "Threatning mails to TE persons, in case of non receipt of DPR timely (1 pm daily)
9. Counts total number of reminders sent to each GA and maintain the delay track date wise
10. Sends the mail in Hindi with hindi numeral dates
11. Now it checks for Abnormally high value of laying/Welding/Conversion/Riser/Meter (update 13.09.2023)
12. It generates warning messages for cases where: charging>laying>welding and also for conversion>meter>riser (update 13.09.2023)
13. It now warns about any revision in DPR (provided that the name of the file remains the same) --> If this breaks, revert to the older update from mail. 
14. Speaks up the warning portion (under trial)


### [use the 3.0.1.7[stable- no reminders / 3.0.1.8 [stable- no count of reminders] in case it breaks: Mail]

In [1]:
####################################################################
__author__ = 'Nishant Sinha'                                       # 
__version__ = '3.0.1.13'                                           #
__maintainer__ = 'Nishant Sinha'                                   #
__status__ = 'Daily Progerss Report Automation'                    #
####################################################################

**Change the path to where all files are present:**

In [2]:
%cd C:\Users\hp\Documents\Python Scripts

C:\Users\hp\Documents\Python Scripts


In [3]:
import numpy as np
import pandas as pd
import os
import shutil
import warnings
from styleframe import StyleFrame
from datetime import datetime, timedelta
import win32com.client
from tqdm.notebook import tqdm
import sys
from time import sleep,time,ctime,strftime
column_settings=[]
import pickle
from colorama import Fore, Style, Back
import imaplib
import base64
import email
import smtplib
import shutil
import subprocess
import win32con
import win32gui
from collections import Counter
import pyttsx3

from tkinter import *
import tkinter.messagebox

from email.message import EmailMessage
from email import encoders

from PIL import ImageGrab
import psutil

from getpass import getpass
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
pd.set_option('display.max_columns',None)

# DATE_IN_DPR= input("Enter the string mentioned in today's progress column in DPR obtained: ")
DATE_IN_DPR= 'today'

recipients= ['nishant@energyworld.biz']
cc= ['nishantsinha.repl@outlook.com']

send_for_reporting= ['lokeshchaudhary.repl@outlook.com']  #send Data Bank and site DPRs with progress date of 15th and last day of month.
mail_dict= {"test_mail":['nishantjin05@gmail.com','nishantsinha0502@gmail.com'],
"mail_varanasi": ['anand.tiwaridatt@gmail.com','ambuj@energyworld.biz','atulpathak.repl@outlook.com','lokeshchaudhary.repl@outlook.com','nitin@energyworld.biz'],
"mail_bhubaneshwar": ['virukumar584@gmail.com','niranjan@energyworld.biz','atulpathak.repl@outlook.com','lokeshchaudhary.repl@outlook.com','nitin@energyworld.biz'],
"mail_patna": ['bhakhilesh.er@gmail.com','santosh@energyworld.biz','atulpathak.repl@outlook.com','lokeshchaudhary.repl@outlook.com','nitin@energyworld.biz'],
"mail_ranchi": ['satyaprakash.singh496@gmail.com','abhijeet@energyworld.biz','atulpathak.repl@outlook.com','lokeshchaudhary.repl@outlook.com','nitin@energyworld.biz'],
"mail_jamshedpur": ['akash.repl@outlook.com','atulpathak.repl@outlook.com','lokeshchaudhary.repl@outlook.com','nitin@energyworld.biz'],
"mail_cuttack": ['sibruta@gmail.com','sharvan@energyworld.biz','atulpathak.repl@outlook.com','lokeshchaudhary.repl@outlook.com','nitin@energyworld.biz']}

# mail_dict= {"test_mail":['nishant.sinha@tractebel.engie.com','nishantjin05@gmail.com','nishantsinha0502@gmail.com']}

vd_count,bd_count,pd_count,rd_count,jd_count,cd_count=0,0,0,0,0,0

pickle_dpr_reminder_count= open("dpr_reminder_count.pickle","rb")
vd=pickle.load(pickle_dpr_reminder_count)
vd_rem_count,bd_rem_count,pd_rem_count,rd_rem_count,jd_rem_count,cd_rem_count=vd.iloc[0,0],vd.iloc[0,1],vd.iloc[0,2],vd.iloc[0,3],vd.iloc[0,4],vd.iloc[0,5]
pickle_dpr_reminder_count.close()

#### If DPR reminder count is messed up somehow, run me to reset:

In [4]:

# dpr_reminder_count=pd.DataFrame(data={'varanasi':[0],'bhubaneshwar':[0],'patna':[0],'ranchi':[0],'jamshedpur':[0],'cuttack':[0]})
# pickle_dpr_reminder_count= open("dpr_reminder_count.pickle","wb")
# pickle.dump(dpr_reminder_count,pickle_dpr_reminder_count)
# pickle_dpr_reminder_count.close()


### Taking Backup of files in workplace (this will have previous day's data):

In [5]:
###### Trial GUI BASED INPUT PROMPT

# import os
# import shutil
# from tkinter import Tk, Label, Button

# def on_button_click():
#     try:
#         os.mkdir("C:\\Users\\hp\\Desktop\\backup_of_python_script_folder")
#     except FileExistsError:
#         pass
#     shutil.rmtree("C:\\Users\\hp\\Desktop\\backup_of_python_script_folder")
#     shutil.copytree("C:\\Users\\hp\\Documents\\Python Scripts","C:\\Users\\hp\\Desktop\\backup_of_python_script_folder")
#     label.config(text="Backup created successfully")

# root = Tk()
# root.title("Backup Python Script Folder")

# label = Label(root, text="Do you want to take a backup of 'Python Script folder?")
# label.pack()

# button = Button(root, text="Yes", command=on_button_click)
# button.pack()

# root.mainloop()

In [6]:
#### Previous state of Python Script folder (irrespective of user_input)

try:
    os.mkdir("C:\\Users\\hp\\Pictures\\mand_back_py_script_folder")
except FileExistsError:
    pass
shutil.rmtree("C:\\Users\\hp\\Pictures\\mand_back_py_script_folder")
shutil.copytree("C:\\Users\\hp\\Documents\\Python Scripts","C:\\Users\\hp\\Pictures\\mand_back_py_script_folder")

#### Conscious decision to take backup of the previous stage of Python Script folder

user_input= input("Do you want to take a backup of 'Python Script folder? Press y/Y to continue'")
if user_input in ['y','Y']:
    try:
        os.mkdir("C:\\Users\\hp\\Desktop\\backup_of_python_script_folder")
    except FileExistsError:
        pass
    shutil.rmtree("C:\\Users\\hp\\Desktop\\backup_of_python_script_folder")
    shutil.copytree("C:\\Users\\hp\\Documents\\Python Scripts","C:\\Users\\hp\\Desktop\\backup_of_python_script_folder")
else:
    pass

Do you want to take a backup of 'Python Script folder? Press y/Y to continue'y


### Booster Cell below:

In [7]:
## Booster cells
boost_up= input("Do you want to boost up the DPR creation process? Press y/Y to boost, any other key to ignore.\n")
if boost_up in ['y','Y']:
    os.system('start excel.exe "C:\\Users\\hp\\Pictures\\booster_for_dpr.xlsx"') # included to boost up the process
else:
    pass

Do you want to boost up the DPR creation process? Press y/Y to boost, any other key to ignore.
n


### Cleaning the folder:

In [8]:
old_file_name_has= ['BHUBANESHWAR','CUTTACK','JAMSHEDPUR','PATNA','RANCHI','VARANASI']
old_folder_name_has= ['COMBINED','OVERSIMPLIFIED','SIMPLIFIED']
for j in old_file_name_has:
    for i in os.listdir():
        try:
            if i.lower().split('.')[1] !='pickle':
                if (i.lower().split('_')[0] in j.lower()):
                    os.remove(i)
        except IndexError:
            continue
try:
    for j in old_folder_name_has:
        for i in os.listdir():
            if i.lower().split('_')[0] in j.lower():
                shutil.rmtree(os.getcwd()+'\\'+i)
except:
    pass
    

### Download from the mail and place it in folder

In [9]:
# # email_user = input('Enter your email ID: ')
# email_user= 'gailcgdwork@gmail.com'
# # email_pass = getpass('Enter the password: ')
# email_pass= '<YOUR PASSWORD HERE>'

# mail = imaplib.IMAP4_SSL('imap.gmail.com',993)
# mail.login(email_user, email_pass)

# mail.select('INBOX')
# typ, data = mail.search(None, 'ALL')
# mail_ids = data[0]
# id_list = mail_ids.split()

# for num in data[0].split():
#     typ, data = mail.fetch(num, '(RFC822)' )
#     raw_email = data[0][1]
#     raw_email_string = raw_email.decode('utf-8')
#     email_message = email.message_from_string(raw_email_string)
#     for part in email_message.walk():
        
#         if part.get_content_maintype() == 'multipart':
#             continue
#         if part.get('Content-Disposition') is None:
#             continue
#         fileName = part.get_filename()
#         if bool('.xlsm' in fileName):
            
#             filePath = os.path.join("C:\\Users\\hp\\Documents\\Python Scripts\\", fileName)
#             if not os.path.isfile(filePath):
#                 fp = open(filePath, 'wb')
#                 fp.write(part.get_payload(decode=True))
#                 fp.close()
#             subject = str(email_message).split("Subject: ", 1)[1].split("\nTo:", 1)[0]
#     mail.store(num,'+FLAGS','\\Deleted')
# mail.expunge()
# mail.close()
# mail.logout()
# print(f'{Fore.CYAN} {Style.BRIGHT} Check the "Python Scripts" folder Nishant! {Fore.RESET} {Style.RESET_ALL}')


###########################################################################################################
#***************** If the code below breaks, switch to the earlier version commented above ****************
###########################################################################################################

list_of_files_downloaded=[]
# email_user = input('Enter your email ID: ')
email_user= 'gailcgdwork@gmail.com'
# email_pass = getpass('Enter the password: ')
email_pass= '<YOUR PASSWORD HERE>'

mail = imaplib.IMAP4_SSL('imap.gmail.com',993)
mail.login(email_user, email_pass)

mail.select('INBOX')
typ, data = mail.search(None, 'ALL')
mail_ids = data[0]
id_list = mail_ids.split()

for num in data[0].split():
    typ, data = mail.fetch(num, '(RFC822)' )
    raw_email = data[0][1]
    raw_email_string = raw_email.decode('utf-8')
    email_message = email.message_from_string(raw_email_string)
    for part in email_message.walk():
        
        if part.get_content_maintype() == 'multipart':
            continue
        if part.get('Content-Disposition') is None:
            continue
        fileName = part.get_filename()
        fileName = fileName.replace(" ", "_").replace("-", "_") #added to avoid issues with blank spaces and "."
        if bool('.xlsm' in fileName):
            list_of_files_downloaded.append(fileName)
            filePath = os.path.join("C:\\Users\\hp\\Documents\\Python Scripts\\", fileName)  
            if not os.path.isfile(filePath):
                fp = open(filePath, 'wb')
                fp.write(part.get_payload(decode=True))
                fp.close()
            subject = str(email_message).split("Subject: ", 1)[1].split("\nTo:", 1)[0]
    mail.store(num,'+FLAGS','\\Deleted')
mail.expunge()
mail.close()
mail.logout()

print(f'{Fore.CYAN} {Style.BRIGHT} Check the "Python Scripts" folder Nishant! {Fore.RESET} {Style.RESET_ALL}')

d =  Counter(list_of_files_downloaded)  
duplicacy_check = [k for k, v in d.items() if v > 1]

if len(duplicacy_check)>0:
    root = Tk()
    tkinter.messagebox.showinfo(title= "Duplicacy Alert!!!", message=f"Check and Delete the duplicate DPR from {duplicacy_check}")
    root.destroy()



  Check the "Python Scripts" folder Nishant!  


### For Construction DPR New implementation

In [10]:
mand_files = ['CGD', 'DF', 'DPR']
run_only_if_all_files_are_present = 0
new_files = []

for j in old_file_name_has:
    for i in os.listdir():
        if i.lower().split('_')[0] in j.lower() and i.lower().split('_')[0] not in mand_files:
            if i.lower().split('.')[1] != 'pickle':
                print(f'{Fore.MAGENTA} {Style.BRIGHT} {i} found {Fore.RESET} {Style.RESET_ALL}')
                if "varanasi".lower() in i.lower().split('.')[0]:
                    remember_me_1 = i
                    with open(i, 'r') as file_handle:
                        pass
                    os.rename(i, "Varanasi_DPR.xlsm")
                if "bhubaneshwar".lower() in i.lower().split('.')[0]:
                    remember_me_2 = i
                    with open(i, "r") as file_handle:
                        pass
                    os.rename(i, "Bhubaneshwar_DPR.xlsm")
                if "patna".lower() in i.lower().split('.')[0]:
                    remember_me_3 = i
                    with open(i, "r") as file_handle:
                        pass
                    os.rename(i, "Patna_DPR.xlsm")
                if "ranchi".lower() in i.lower().split('.')[0]:
                    remember_me_4 = i
                    with open(i, "r") as file_handle:
                        pass
                    os.rename(i, "Ranchi_DPR.xlsm")
                if "jamshedpur".lower() in i.lower().split('.')[0]:
                    remember_me_5 = i
                    with open(i, "r") as file_handle:
                        pass
                    os.rename(i, "Jamshedpur_DPR.xlsm")
                if "cuttack".lower() in i.lower().split('.')[0]:
                    remember_me_6 = i
                    with open(i, "r") as file_handle:
                        pass
                    os.rename(i, "Cuttack_DPR.xlsm")
                new_files.append(i)

new_file_list = []
for i in new_files:
    new_file_list.append(i.split('_')[0])

  BHUBANESHWAR_DPR_P_06.01.2024____.xlsm found  
  Cuttack_DPR__06.01.2024.xlsm found  
  Jamshedpur_DPR_06_01_2024.xlsm found  
  Patna_DPR_1105_P_06.01.2024__.xlsm found  
  Ranchi_DPR_06.01.2024.xlsm found  
  Varanasi_DPR_06.01.2024.xlsm found  


### Check if all the files from all the sites have been received:

In [11]:
try:
    if (not pd.DataFrame(new_file_list).duplicated().any()) & (pd.DataFrame(new_file_list).count()==6)[0]:
        run_only_if_all_files_are_present=1
except IndexError:
    print(f"{Fore.BLACK}{Back.RED}No site DPR file present in the folder 'Python Scripts' folder!!!{Fore.RESET}{Back.RESET}")

### Files from which GAs are received, which are not?

In [12]:
mail_to_gas = []
new_file_list_lower=[]
for i in new_file_list:
    new_file_list_lower.append(i.lower())
    
new_file_list=new_file_list_lower

for i in new_file_list:
    print(f"{Fore.GREEN} {Style.BRIGHT} DPR from {i.upper()} received  {Fore.RESET} {Style.RESET_ALL}")
    
for i in old_file_name_has:
    if i.lower() not in new_file_list:
        mail_to_gas.append(i)
        if i.lower()=='varanasi':
            vd_count=1
            vd_rem_count+=1
        if i.lower()=='bhubaneshwar':
            bd_count=1
            bd_rem_count+=1
        if i.lower()=='patna':
            pd_count=1
            pd_rem_count+=1
        if i.lower()=='ranchi':
            rd_count=1
            rd_rem_count+=1
        if i.lower()=='jamshedpur':
            jd_count=1
            jd_rem_count+=1
        if i.lower()=='cuttack':
            cd_count=1
            cd_rem_count+=1
        
        if datetime.today().time().hour<13:
            print(f"{Fore.RED} {Style.BRIGHT} DPR from {i.upper()} not received. Not sending mail yet, since decided time is 1 PM.{Fore.RESET} {Style.RESET_ALL}")
        elif datetime.today().time().hour>=13:
            print(f"{Fore.RED} {Style.BRIGHT} DPR from {i.upper()} not received. Sending reminder mail.{Fore.RESET} {Style.RESET_ALL}")
            
dpr_delay_in = open("dpr_delay_count.pickle",'rb')
dpr_delay_count= pickle.load(dpr_delay_in)
if dpr_delay_count.iloc[-1,0]!=datetime.now().date() and datetime.today().time().hour>=13:
#     dpr_delay_count=dpr_delay_count.append({'day':datetime.now().date(),'varanasi':vd_count,'bhubaneshwar':bd_count,'patna':pd_count,'ranchi':rd_count,'jamshedpur':jd_count,'cuttack':cd_count},ignore_index=True)
    dpr_delay_count = pd.concat([dpr_delay_count, pd.DataFrame({'day':datetime.now().date(),'varanasi':vd_count,'bhubaneshwar':bd_count,'patna':pd_count,'ranchi':rd_count,'jamshedpur':jd_count,'cuttack':cd_count}, index=[0])], ignore_index=True)
    dpr_delay_count.drop_duplicates(inplace=True)
    pickle_dpr_delay_count= open("dpr_delay_count.pickle","wb")
    pickle.dump(dpr_delay_count,pickle_dpr_delay_count)
    pickle_dpr_delay_count.close()  
dpr_delay_in.close()

if run_only_if_all_files_are_present!=1:
    dpr_reminder_count=pd.DataFrame(data={'varanasi':[vd_rem_count],'bhubaneshwar':[bd_rem_count],'patna':[pd_rem_count],'ranchi':[rd_rem_count],'jamshedpur':[jd_rem_count],'cuttack':[cd_rem_count]})
    pickle_dpr_reminder_count= open("dpr_reminder_count.pickle","wb")
    pickle.dump(dpr_reminder_count,pickle_dpr_reminder_count)
    pickle_dpr_reminder_count.close()
    
if run_only_if_all_files_are_present==1:
    dpr_reminder_count=pd.DataFrame(data={'varanasi':[0],'bhubaneshwar':[0],'patna':[0],'ranchi':[0],'jamshedpur':[0],'cuttack':[0]})
    pickle_dpr_reminder_count= open("dpr_reminder_count.pickle","wb")
    pickle.dump(dpr_reminder_count,pickle_dpr_reminder_count)
    pickle_dpr_reminder_count.close()
    
if run_only_if_all_files_are_present==1:
    if dpr_delay_count.iloc[-1,0]!=datetime.now().date() and datetime.today().time().hour<13:
        dpr_delay_in = open("dpr_delay_count.pickle",'rb')
        dpr_delay_count= pickle.load(dpr_delay_in)
#         dpr_delay_count=dpr_delay_count.append({'day':datetime.now().date(),'varanasi':0,'bhubaneshwar':0,'patna':0,'ranchi':0,'jamshedpur':0,'cuttack':0},ignore_index=True)
        dpr_delay_count = pd.concat([dpr_delay_count, pd.DataFrame({'day':datetime.now().date(),'varanasi':0,'bhubaneshwar':0,'patna':0,'ranchi':0,'jamshedpur':0,'cuttack':0}, index=[0])], ignore_index=True)
        dpr_delay_count.drop_duplicates(inplace=True)
        pickle_dpr_delay_count= open("dpr_delay_count.pickle","wb")
        pickle.dump(dpr_delay_count,pickle_dpr_delay_count)
        pickle_dpr_delay_count.close()  
        dpr_delay_in.close()
        

  DPR from BHUBANESHWAR received   
  DPR from CUTTACK received   
  DPR from JAMSHEDPUR received   
  DPR from PATNA received   
  DPR from RANCHI received   
  DPR from VARANASI received   


### Generating REMINDER for non-receipt of DPR timely.

In [13]:
reminder_number=0
def threatning_mail(ga_name):
    if datetime.today().time().hour>12:
        ghante= datetime.today().time().hour-12
    else:
        ghante= datetime.today().time().hour
    minute= datetime.today().minute

    msg= EmailMessage()
    sender= 'gailcgdwork@gmail.com'
    msg['From']=sender
    total_tracked_days=dpr_delay_count.shape[0]
    
    if ga_name.lower()== 'varanasi':
        reminder_number= vd_rem_count
        total_delays= dpr_delay_count.varanasi.sum()
        msg['To']= mail_dict["mail_varanasi"][0]
        msg['CC']= mail_dict["mail_varanasi"][1:]
    if ga_name.lower()== 'bhubaneshwar':
        reminder_number= bd_rem_count
        total_delays=dpr_delay_count.bhubaneshwar.sum()
        msg['To']= mail_dict["mail_bhubaneshwar"][0]
        msg['CC']= mail_dict["mail_bhubaneshwar"][1:]
    if ga_name.lower()== 'patna':
        reminder_number= pd_rem_count
        total_delays=dpr_delay_count.patna.sum()
        msg['To']= mail_dict["mail_patna"][0]
        msg['CC']= mail_dict["mail_patna"][1:]
    if ga_name.lower()== 'ranchi':
        reminder_number= rd_rem_count
        total_delays=dpr_delay_count.ranchi.sum()
        msg['To']= mail_dict["mail_ranchi"][0]
        msg['CC']= mail_dict["mail_ranchi"][1:]
    if ga_name.lower()== 'jamshedpur':
        reminder_number= jd_rem_count
        total_delays=dpr_delay_count.jamshedpur.sum()
        msg['To']= mail_dict["mail_jamshedpur"][0]
        msg['CC']= mail_dict["mail_jamshedpur"][1:]
    if ga_name.lower()== 'cuttack':
        reminder_number= cd_rem_count
        total_delays=dpr_delay_count.cuttack.sum()
        msg['To']= mail_dict["mail_cuttack"][0]
        msg['CC']= mail_dict["mail_cuttack"][1:]

#     if ga_name.lower()== 'varanasi':
#         reminder_number= vd_rem_count
#         total_delays= dpr_delay_count.varanasi.sum()
#         msg['To']= mail_dict["test_mail"][0]
#         msg['CC']= mail_dict["test_mail"][1:]
#     if ga_name.lower()== 'bhubaneshwar':
#         reminder_number= bd_rem_count
#         total_delays=dpr_delay_count.bhubaneshwar.sum()
#         msg['To']= mail_dict["test_mail"][0]
#         msg['CC']= mail_dict["test_mail"][1:]
#     if ga_name.lower()== 'patna':
#         reminder_number= pd_rem_count
#         total_delays=dpr_delay_count.patna.sum()
#         msg['To']= mail_dict["test_mail"][0]
#         msg['CC']= mail_dict["test_mail"][1:]
#     if ga_name.lower()== 'ranchi':
#         reminder_number= rd_rem_count
#         total_delays=dpr_delay_count.ranchi.sum()
#         msg['To']= mail_dict["test_mail"][0]
#         msg['CC']= mail_dict["test_mail"][1:]
#     if ga_name.lower()== 'cuttack':
#         reminder_number= cd_rem_count
#         total_delays=dpr_delay_count.cuttack.sum()
#         msg['To']= mail_dict["test_mail"][0]
#         msg['CC']= mail_dict["test_mail"][1:]
#     if ga_name.lower()== 'jamshedpur':
#         reminder_number= jd_rem_count
#         total_delays=dpr_delay_count.jamshedpur.sum()
#         msg['To']= mail_dict["test_mail"][0]
#         msg['CC']= mail_dict["test_mail"][1:]

        
    msg['Subject']= f"{ga_name.upper()}: AUTO-REMINDER - {reminder_number} FOR NON-RECEIPT OF DPR !!!"
    msg.set_content(f"""REMINDER #{reminder_number} FOR DPR!!!\n\nIt is {datetime.today().strftime("%I:%M %p")} now. Please send the DPR timely.\n\nThe DPR reporting yesterday's progress has been delayed by {ghante-1} hours and {minute} minutes from the decided time of 1 PM. Please Send.

NOTE: 
\n1. Total days ("recorded"), when your GA delayed in sending DPR since 24th July 2022 is: {total_delays} days out of {total_tracked_days} days.
\n2. For now, this mail is only being sent internally. In case of repeated delays in receipt of DPR from your site, it shall be sent to your GAIL GA incharge as well.
\n3. DO NOT reply to this mail. This mail is meant to receive only your Site DPRs.
 \n\n 
IF YOU BELIEVE THIS MAIL REACHED YOU IN ERROR, PLEASE CONTACT THE UNDERSIGNED:
 \n
Regards,
Nishant Sinha
Deputy Manager
  
  """)
    with smtplib.SMTP_SSL('smtp.gmail.com',465) as smtp:
            smtp.login('gailcgdwork@gmail.com', '<YOUR PASSWORD HERE>')
            smtp.send_message(msg)
    print(f"Reminder Mail sent successfully to {Fore.BLUE} {Style.BRIGHT}{ga_name}{Fore.RESET} {Style.RESET_ALL}")
 
   
if mail_to_gas:
    for ga in mail_to_gas:
        if datetime.today().time().hour>=13:  #### to do unpickle the dpr_delay_count (check once, check for sum, add another mail function and call it if this value is greater than say 50)
            threatning_mail(ga)
        else:
            pass
    if datetime.today().time().hour<13:
        dpr_reminder_count=pd.DataFrame(data={'varanasi':[0],'bhubaneshwar':[0],'patna':[0],'ranchi':[0],'jamshedpur':[0],'cuttack':[0]})
        pickle_dpr_reminder_count= open("dpr_reminder_count.pickle","wb")
        pickle.dump(dpr_reminder_count,pickle_dpr_reminder_count)
        pickle_dpr_reminder_count.close()
        
        print(f"{Fore.YELLOW} {Style.BRIGHT}Not sending any reminder for now, as fixed time is 1 PM everyday{Fore.RESET} {Style.RESET_ALL}")
        


### Re-establish the state of files, if all files were not received.

In [14]:
def restore_state():
    msg = EmailMessage()
    sender = 'gailcgdwork@gmail.com'
    msg['From'] = sender
    msg['To'] = sender
    msg['Subject'] = 'DPR being resent (not all DPRs received from sites/ Error Occured)'
    for i in os.listdir():
        for j in old_file_name_has:
            if j.lower() in i.lower() and '.xlsm' in i:
                if j.lower() == i.split('_')[0].lower():
                    try:
                        if j.lower() == 'varanasi':
                            with open("C:\\Users\\hp\\Documents\\Python Scripts\\Varanasi_DPR.xlsm", 'r') as file_handle:
                                pass
                            os.rename("C:\\Users\\hp\\Documents\\Python Scripts\\Varanasi_DPR.xlsm", f"C:\\Users\\hp\\Documents\\Python Scripts\\{remember_me_1}")
                        elif j.lower() == 'bhubaneshwar':
                            with open("C:\\Users\\hp\\Documents\\Python Scripts\\Bhubaneshwar_DPR.xlsm", 'r') as file_handle:
                                pass
                            os.rename("C:\\Users\\hp\\Documents\\Python Scripts\\Bhubaneshwar_DPR.xlsm", f"C:\\Users\\hp\\Documents\\Python Scripts\\{remember_me_2}")
                        elif j.lower() == 'patna':
                            with open("C:\\Users\\hp\\Documents\\Python Scripts\\Patna_DPR.xlsm", 'r') as file_handle:
                                pass
                            os.rename("C:\\Users\\hp\\Documents\\Python Scripts\\Patna_DPR.xlsm", f"C:\\Users\\hp\\Documents\\Python Scripts\\{remember_me_3}")
                        elif j.lower() == 'ranchi':
                            with open("C:\\Users\\hp\\Documents\\Python Scripts\\Ranchi_DPR.xlsm", 'r') as file_handle:
                                pass
                            os.rename("C:\\Users\\hp\\Documents\\Python Scripts\\Ranchi_DPR.xlsm", f"C:\\Users\\hp\\Documents\\Python Scripts\\{remember_me_4}")
                        elif j.lower() == 'jamshedpur':
                            with open("C:\\Users\\hp\\Documents\\Python Scripts\\Jamshedpur_DPR.xlsm",'r') as file_handle:
                                pass
                            os.rename("C:\\Users\\hp\\Documents\\Python Scripts\\Jamshedpur_DPR.xlsm",f"C:\\Users\\hp\\Documents\\Python Scripts\\{remember_me_5}")
                        elif j.lower() == 'cuttack':
                            with open("C:\\Users\\hp\\Documents\\Python Scripts\\Cuttack_DPR.xlsm",'r') as file_handle:
                                pass
                            os.rename("C:\\Users\\hp\\Documents\\Python Scripts\\Cuttack_DPR.xlsm",f"C:\\Users\\hp\\Documents\\Python Scripts\\{remember_me_6}")
                    except (PermissionError,FileNotFoundError):
                        pass
    for i in os.listdir():
        for j in old_file_name_has:
            if j.lower() in i.lower() and '.xlsm' in i:
                with open(i, 'rb') as f:
                    file_data= f.read()
                    file_name= f.name   
                    f.close() ###
                msg.add_attachment(file_data,maintype='application',subtype='octet-stream',filename=file_name)
                
    with smtplib.SMTP_SSL('smtp.gmail.com',465) as smtp:
        smtp.login('gailcgdwork@gmail.com', '<YOUR PASSWORD HERE>')
        smtp.send_message(msg)
    os.chdir('C:\\Users\\hp\\Documents')
    
    shutil.copy("C:\\Users\\hp\\Documents\\Python Scripts\\dpr_delay_count.pickle","C:\\Users\\hp\\Desktop\\backup_of_python_script_folder")
    shutil.copy("C:\\Users\\hp\\Documents\\Python Scripts\\dpr_reminder_count.pickle","C:\\Users\\hp\\Desktop\\backup_of_python_script_folder")
    
    shutil.copy("C:\\Users\\hp\\Documents\\Python Scripts\\dpr_delay_count.pickle","C:\\Users\\hp\\Pictures\\mand_back_py_script_folder")
    shutil.copy("C:\\Users\\hp\\Documents\\Python Scripts\\dpr_reminder_count.pickle","C:\\Users\\hp\\Pictures\\mand_back_py_script_folder")

    
    shutil.rmtree("C:\\Users\\hp\\Documents\\Python Scripts")
    if user_input in ['y','Y']:
        shutil.copytree("C:\\Users\\hp\\Desktop\\backup_of_python_script_folder","C:\\Users\\hp\\Documents\\Python Scripts")
        print(f'The backup from {Fore.GREEN}"CONSCIOUSLY" {Fore.RESET}saved state has been restored')
    else:
        shutil.copytree("C:\\Users\\hp\\Pictures\\mand_back_py_script_folder","C:\\Users\\hp\\Documents\\Python Scripts")  
        print(f'The backup from {Fore.YELLOW} "UNCONSCIOUSLY" {Fore.RESET}saved state has been restored')
    os.chdir("C:\\Users\\hp\\Documents\\Python Scripts")
    
if run_only_if_all_files_are_present!=1:
    restore_state()
else:
    pass


In [15]:
# import psutil
# import os

# def kill_process_using_file(file_path):
#     for proc in psutil.process_iter():
#         try:
#             files = proc.open_files()
#             for f in files:
#                 if f.path == file_path:
#                     proc.kill()
#                     break
#         except Exception:
#             pass

# file_path = "C:\\Users\\hp\\Documents\\Python Scripts\\Varanasi_DPR.xlsm"
# kill_process_using_file(file_path)


### Loading the Data Bank 

In [16]:
# ##### Uncomment the following and run for any changes made to the Data Bank file, then comment again ###########

# # Note: First convert the xlsb file to xlsm file

# df_main = pd.read_excel("CGD_DPR_DATA_BANK.xlsm",sheet_name="Data")
# df_main.columns= df_main.iloc[0]
# df_main= df_main[1:]

# pickle_out= open("df_main.pickle","wb")
# pickle.dump(df_main,pickle_out)
# pickle_out.close()

In [17]:
pickle_in = open("df_main.pickle",'rb')
df_main= pickle.load(pickle_in)
pickle_in.close()

### DPR engine

In [18]:
mismatch_track=0
if (run_only_if_all_files_are_present==1) and (len(duplicacy_check) == 0):
    final_combined_df = pd.DataFrame()
    must_contain_string= ['riser','meter','20','32','63','90','125','conversion','charged','steel_4','steel_6','steel_8','steel_12','weld_4','weld_6','weld_8','weld_12','comm_4','comm_6','comm_8','comm_12']  

    try:
        os.mkdir("OVERSIMPLIFIED_FILES")
    except FileExistsError:
        pass
    try:
        os.mkdir("SIMPLIFIED_FILES")
    except FileExistsError:
        pass
    try:
        os.mkdir("COMBINED_DPR")
    except FileExistsError:
        pass

    
    def for_steel_sheet():
        column_stop_here= ''
        global df2_steel,df_steel
        df_steel= pd.read_excel(file_name,sheet_name=sheet_name)
        df_steel= df_steel.iloc[df_steel[df_steel['Unnamed: 3'].str.contains('Progress',case=False,regex=True)==True].index.min():,:]
        df_steel.drop([3,5],inplace=True)                               #----------------> Be cautious here
        df_steel.iloc[0]=df_steel.iloc[0].combine_first(df_steel.iloc[1])
        df_steel.columns= df_steel.iloc[0]

        lower_index= np.flatnonzero(df_steel['Activity Description'].dropna().str.contains("welding",case=False))[0]+1
        upper_index= np.flatnonzero(df_steel['Activity Description'].dropna().str.contains("welding",case=False))[0]+6
        df_steel_1= df_steel.iloc[lower_index: upper_index]

        lower_index= np.flatnonzero(df_steel['Activity Description'].dropna().str.contains("lowering",case=False))[0]+1
        upper_index= np.flatnonzero(df_steel['Activity Description'].dropna().str.contains("lowering",case=False))[0]+6
#         df_steel_1= df_steel_1.append(df_steel.iloc[lower_index: upper_index])
        df_steel_1 = pd.concat([df_steel_1, df_steel.iloc[lower_index: upper_index]], ignore_index=True)

    
        lower_index= np.flatnonzero(df_steel['Activity Description'].dropna().str.contains("Commissioning",case=False))[0]+1 ### *(*(*()))
        upper_index= np.flatnonzero(df_steel['Activity Description'].dropna().str.contains("Commissioning",case=False))[0]+6 ### *(*(*()))
#         df_steel_1= df_steel_1.append(df_steel.iloc[lower_index: upper_index])  ### *(*(*()))
        df_steel_1 = pd.concat([df_steel_1, df_steel.iloc[lower_index: upper_index]], axis=0, ignore_index=True)
        df_steel= df_steel_1
        ################################################# OR  #######################################################
        # lower_index=df_steel['Activity Description'].dropna().str.contains("welding",case=False).index[df_steel['Activity Description'].dropna().str.contains("welding",case=False)][0]
        # upper_index=df_steel['Activity Description'].dropna().str.contains("welding",case=False).index[df_steel['Activity Description'].dropna().str.contains("welding",case=False)][0]+4
        # df_steel.loc[lower_index:upper_index,:]
        #############################################################################################################
        for x in df_steel.columns.unique():
            if "scope" in str(x).lower():                         #----------------> Last word must have "inch" somewhere
                    column_stop_here = x
            else:
                pass

        df_steel= df_steel.loc[:,: column_stop_here]
        df_steel= df_steel.set_index(df_steel.iloc[:,1])
        df_steel=df_steel.iloc[:,2:df_steel.shape[1]-1]

        df_steel.columns.name= "Column_Description"

        df1_steel= df_steel.iloc[:,df_steel.columns=="today"].iloc[:,0:]
        df1_steel.replace(0, np.nan, inplace=True)
        df1_steel.columns= df_steel.columns[1::4][0:df1_steel.columns.shape[0]]

        df2_steel= df1_steel.loc[:,~df1_steel.isna().all()].T
        df2_steel.dropna(axis=1, how='all',inplace=True)
        df2_steel= df2_steel.iloc[:,np.flatnonzero(df2_steel.columns.str.contains('weld_|steel_|comm_',case=False))]
        return df2_steel,df_steel

    def for_mdpe_sheet():
        column_stop_here= ''
        global df2,df
        df= pd.read_excel(file_name,sheet_name=sheet_name)
        df= df.iloc[df[df['Unnamed: 1'].str.contains('Description|Activity',case=False,regex=True)==True].index.min():,:]
        df.drop([4,5],inplace=True)                               #----------------> Be cautious here
        df.iloc[0]=df.iloc[0].combine_first(df.iloc[1])
        df.columns= df.iloc[0]
        df.drop([3,6,7,13,17,19,20], inplace=True)    ### *(*(*(           #----------------> Be cautious here... 20 included for on gas 
        row_stop_here= df.iloc[:,:3].dropna(thresh=3).shape[0]   #----------------> assuming first 3 values are not blank
        for i in df.columns.unique():
            if "inch" in str(i).lower():                         #----------------> Last word must have "inch" somewhere
                    column_stop_here = i
            else:
                pass

        df= df.loc[:,: column_stop_here]
        df= df.iloc[0:row_stop_here,1:]

    #                 df= df.iloc[3:,1:]
    #                 df.drop([4,5],inplace=True)
    #                 df.iloc[0]= df.iloc[0].combine_first(df.iloc[1])
    #                 df.columns= df.iloc[0]
    #                 df.drop([3,6,7,13,14,17,19], inplace=True)
    #                 df= df.loc[:18,: ' INCH-KM']


        df= df.set_index(df.iloc[:,0])
        df.drop(df.columns[0],axis=1,inplace=True)
        df.columns.name= "Column_Description"

    ###############################################################################

        df1= df.iloc[:,df.columns==DATE_IN_DPR].iloc[:,0:]
        df1.replace(0, np.nan, inplace=True)
        df1.columns= df.columns[1::4][0:df1.columns.shape[0]]

    ###############################################################################

        df2= df1.loc[:,~df1.isna().all()].T
        df2.dropna(axis=1, how='all',inplace=True)
        return df2,df



    for file_name in tqdm(os.listdir()):
        file_name_lower= file_name.lower()
        condition1= '.xlsx' in file_name_lower or '.xls' in file_name_lower or ".xlsm" in file_name_lower
        condition2= "~" not in file_name_lower
        condition3= "cgd_dpr_data_bank" not in file_name_lower
        condition4= 'nis' not in file_name
        condition5= 'DPR_MACRO' not in file_name_lower
        condition= [condition1,condition2,condition3,condition4, condition5]
        both_sheets_flag= 0
        if all(condition):
            xls = pd.ExcelFile(file_name)
            no_progress_report=0
            for sheet_name in xls.sheet_names:
                if "steel" in sheet_name.lower():                                 #-------------------------> Note here
                    df2_steel,df_steel= for_steel_sheet()
                    
                    # first run me in tino blocks ko comment karna hai (so that initial files generate hon). New contractor add 
                    # hota hai to error aayega (size mismatch). New vendor include karte samay, in tino block ko comment karna hoga
                    ### updated on 26-04-2023 (several vendors added, no error reported even with new vendor inclusion)
              
             ## Block 1
                    df_previous_steel= df_steel.iloc[:,df_steel.columns=='Previous'].iloc[:,0:]
                    df_previous_steel.replace(0, np.nan, inplace=True)
                    df_previous_steel.columns= df_steel.columns[1::4][0:df_previous_steel.columns.shape[0]]  ## aaj ka previous value
            ## Block 2
                    pickle_in_steel = open(f"{file_name.split('_')[0].lower()}_steel.pickle",'rb')
                    df_cumulative_steel= pickle.load(pickle_in_steel) ## beete kal ka cumulative value  
                    pickle_in_steel.close()  ##---##
            ## Block 3
                    mischief_steel= abs(df_cumulative_steel.replace(np.nan,0,inplace=False)-df_previous_steel.replace(np.nan,0,inplace=False)).sum()>0.001
                    mischief_steel=mischief_steel[mischief_steel==True]
                    if mischief_steel.sum()>0:
                        x= abs(df_cumulative_steel[mischief_steel.index]-df_previous_steel[mischief_steel.index])>0
                        for i in range(x.shape[1]):
                            y= x[x.columns[i]]
                        
                        print(f"\n {Back.RED} Probable manipulation noted! Check: {Back.RESET} {Fore.RED} {Style.BRIGHT} {y[y].dropna().index.values} for the vendors: {y.name} in DPR received from {file_name.split('_')[0]} {Fore.RESET} {Style.RESET_ALL}\n")
#                         print(abs((df_cumulative_steel-df_previous_steel).sum()))
                    else:
                        pass
            ## Block 4        
                    
                    df_cumulative_steel= df_steel.iloc[:,df_steel.columns=='Cumulative'].iloc[:,0:]
                    df_cumulative_steel.replace(0, np.nan, inplace=True)
                    df_cumulative_steel.columns= df_steel.columns[1::4][0:df_cumulative_steel.columns.shape[0]]
                    pickle_steel= open(f"{file_name.split('_')[0].lower()}_steel.pickle","wb")
                    pickle.dump(df_cumulative_steel,pickle_steel)
                    pickle_steel.close()
                              
                    both_sheets_flag+=1
                    continue
                              
                              
                if "mdpe" in sheet_name.lower():                                 #-------------------------> Note here
                    df2,df= for_mdpe_sheet()
                    
                    df_previous_mdpe= df.iloc[:,df.columns=='Previous'].iloc[:,0:]
                    df_previous_mdpe.replace(0, np.nan, inplace=True)
                    df_previous_mdpe.columns= df.columns[1::4][0:df_previous_mdpe.columns.shape[0]]  ## aaj ka previous value

                    pickle_in_mdpe = open(f"{file_name.split('_')[0].lower()}_mdpe.pickle",'rb')
                    df_cumulative_mdpe= pickle.load(pickle_in_mdpe) ## beete kal ka cumulative value
                    pickle_in_mdpe.close()  ##-----##
                    
                    mischief_mdpe= abs((df_cumulative_mdpe.replace(np.nan,0,inplace=False)-df_previous_mdpe.replace(np.nan,0,inplace=False)).sum())>0.001
                    mischief_mdpe=mischief_mdpe[mischief_mdpe==True]
                    if mischief_mdpe.sum()>0:
                        x= abs(df_cumulative_mdpe[mischief_mdpe.index]-df_previous_mdpe[mischief_mdpe.index])>0
                        for i in range(x.shape[1]):
                            y= x[x.columns[i]]
                            print(f"\n {Back.RED} Probable manipulation noted! Check: {Back.RESET}{Fore.RED} {Style.BRIGHT} {y[y].dropna().index.values} for the vendors: {y.name} in DPR received from {file_name.split('_')[0]} {Fore.RESET} {Style.RESET_ALL}\n")
#                         print(abs((df_cumulative_mdpe-df_previous_mdpe).sum()))
                    else:
                        pass
                    
                    
                    df_cumulative_mdpe= df.iloc[:,df.columns=='Cumulative'].iloc[:,0:]
                    df_cumulative_mdpe.replace(0, np.nan, inplace=True)
                    df_cumulative_mdpe.columns= df.columns[1::4][0:df.columns.shape[0]][0:-2]
                    pickle_mdpe= open(f"{file_name.split('_')[0].lower()}_mdpe.pickle","wb")
                    pickle.dump(df_cumulative_mdpe,pickle_mdpe)
                    pickle_mdpe.close()
                              
                              
                    both_sheets_flag+=1
                    continue
                if both_sheets_flag==2:    
                    df2= pd.concat([df2,df2_steel],sort=False)

      ################ try except added, so that with 0 progress, no error comes up ###########              
                    try:
                        df2.loc["Total"]= df2.fillna(0).sum()
                    except ValueError:
                        if no_progress_report<1:
                            print(f"{Fore.RED}{Style.BRIGHT}{Back.GREEN}No progress reported from {file_name.split('_')[0]} ? {Fore.RESET}{Style.RESET_ALL}{Back.RESET}")
                            no_progress_report+=1
                            mismatch_track+=1
                        pass

      ################### if this breaks, delete try and except; keep df2.loc in line with df2= pd.concat ######


                    df2.rename(columns={'Commissioning':'Charged'},inplace=True)
                    df2.to_excel(f"{os.getcwd()}\\SIMPLIFIED_FILES\\{file_name.split('_')[0]}_nis_simplified.xlsx", \
                                      file_name.split('_')[0])
                    final_df= pd.DataFrame()
                    index= 0
                    count=0
                    for city_name in df_main.CITY.unique():
                        if (type(city_name)!=float) and file_name.split("_")[0].lower()==city_name.lower():
                            df_varanasi = df_main[df_main.CITY==city_name]

                            for i in range(df2.shape[0]):
                                df_vns_1= df_varanasi[df_varanasi.CONTRACTOR==df2.iloc[i:i+1].index[0]]
                                for j in range(len(df2.columns[(df2.iloc[i:i+1]!=0).all(axis=0)])):
                                    value= df2.loc[df2.iloc[i:i+1].index[0],df2.columns[(df2.iloc[i:i+1]!=0).all(axis=0)][j]]
                                    if  np.isnan(value).all():
                                        pass
                                    else:
                 #######################################################   Changes made in this part
                ######################## when a vendor is repeated in 2 sheets ###########################
                                        if len(np.array(value).shape)!=0:
                                            for k in value:
                                                if np.isnan(k):
                                                    pass
                                                else:
                 #######################################################                       
                                                    try:
                                                        if k<0:
                                                            print(f"{Fore.YELLOW} {Style.BRIGHT} Please Note: Negative value({k}) entered for {df2.columns[(df2.iloc[i:i+1]!=0).all(axis=0)][j]} of contractor {df2.loc[df2.iloc[i:i+1].index[0]].name} {Fore.RESET} {Style.RESET_ALL} in {city_name} ")
                                                    except IndexError:
                                                        pass
                                                    z = df2.columns[(df2.iloc[i:i+1]!=0).all(axis=0)][j].split(' ')
                                                    z_lower= [n.lower() for n in z] 
                                                    nish=0
                                                    for val in z_lower:
                                                        if val in must_contain_string:
                                                            nish= val
                                                    if_else_run_track=0
                                                    if nish in must_contain_string[9:]:
                                                        nish_1= nish.split('_')[0]
                                                        if nish_1=='comm':
                                                            nish_1='charged'
                                                        nish_2= nish.split('_')[1]
                                                        if df_vns_1.ACTIVITY.str.contains(nish_1,case=False).sum()>0:
                                                            if df_vns_1.SIZE.str.contains(nish_2,case=False).sum()>0:
#                                                                 final_df= final_df.append(df_vns_1[(df_vns_1.ACTIVITY.str.contains(nish_1,case=False) & df_vns_1.SIZE.str.contains(nish_2,case=False))==True].iloc[0:1],ignore_index=True,sort=False)
                                                                final_df = pd.concat([final_df, df_vns_1[(df_vns_1['ACTIVITY'].str.contains(nish_1, case=False) & df_vns_1['SIZE'].str.contains(nish_2, case=False))==True].iloc[0:1]], ignore_index=True, sort=False)
                                                                final_df.iloc[index,7]= k
                                                                index+=1
                                                                if_else_run_track=1


                                                    elif df_vns_1.ACTIVITY.str.contains(nish,case=False).sum()>0:
#                                                         final_df= final_df.append(df_vns_1[df_vns_1.ACTIVITY.str.contains(nish,case=False)==True].iloc[0:1],ignore_index=True,sort=False)
                                                        final_df = pd.concat([final_df, df_vns_1[df_vns_1['ACTIVITY'].str.contains(nish, case=False)==True].iloc[0:1]], ignore_index=True, sort=False)

                                                        final_df.iloc[index,7]= k
                                                        index+=1
                                                        if_else_run_track=1

                                                    elif df_vns_1.SIZE.str.contains(nish,case=False).sum()>0:
#                                                         final_df= final_df.append( df_vns_1[df_vns_1.SIZE.str.contains(nish,case=False)==True].iloc[0:1],ignore_index=True,sort=False)
                                                        final_df = pd.concat([final_df, df_vns_1[df_vns_1['SIZE'].str.contains(nish, case=False)==True].iloc[0:1]], ignore_index=True, sort=False)

                                                        final_df.iloc[index,7]= k
                                                        index+=1
                                                        if_else_run_track=1

                                        else:
                                            try:
                                                if value<0:
                                                    print(f"{Fore.YELLOW} {Style.BRIGHT} Please Note: Negative value({value}) entered for {df2.columns[(df2.iloc[i:i+1]!=0).all(axis=0)][j]} of contractor {df2.loc[df2.iloc[i:i+1].index[0]].name} {Fore.RESET} {Style.RESET_ALL} in {city_name} ")
                                            except IndexError:
                                                pass
                                            z = df2.columns[(df2.iloc[i:i+1]!=0).all(axis=0)][j].split(' ')
                                            z_lower= [n.lower() for n in z] 
                                            nish=0
                                            for val in z_lower:
                                                if val in must_contain_string:
                                                    nish= val
                                            if_else_run_track=0
                                            if nish in must_contain_string[9:]:
                                                nish_1= nish.split('_')[0]
                                                if nish_1=='comm':
                                                    nish_1='charged'
                                                nish_2= nish.split('_')[1]
                                                if df_vns_1.ACTIVITY.str.contains(nish_1,case=False).sum()>0:
                                                    if df_vns_1.SIZE.str.contains(nish_2,case=False).sum()>0:
#                                                         final_df= final_df.append(df_vns_1[(df_vns_1.ACTIVITY.str.contains(nish_1,case=False) & df_vns_1.SIZE.str.contains(nish_2,case=False))==True].iloc[0:1],ignore_index=True,sort=False)
                                                        final_df = pd.concat([final_df, df_vns_1[(df_vns_1['ACTIVITY'].str.contains(nish_1, case=False) & df_vns_1['SIZE'].str.contains(nish_2, case=False))==True].iloc[0:1]], ignore_index=True, sort=False)

                                                        final_df.iloc[index,7]= value
                                                        index+=1
                                                        if_else_run_track=1


                                            elif df_vns_1.ACTIVITY.str.contains(nish,case=False).sum()>0:
#                                                 final_df= final_df.append(df_vns_1[df_vns_1.ACTIVITY.str.contains(nish,case=False)==True].iloc[0:1],ignore_index=True,sort=False)
                                                final_df = pd.concat([final_df, df_vns_1[df_vns_1['ACTIVITY'].str.contains(nish, case=False)==True].iloc[0:1]], ignore_index=True, sort=False)

                                                final_df.iloc[index,7]= value
                                                index+=1
                                                if_else_run_track=1

                                            elif df_vns_1.SIZE.str.contains(nish,case=False).sum()>0:
#                                                 final_df= final_df.append( df_vns_1[df_vns_1.SIZE.str.contains(nish,case=False)==True].iloc[0:1],ignore_index=True,sort=False)
                                                final_df = pd.concat([final_df, df_vns_1[df_vns_1['SIZE'].str.contains(nish, case=False)==True].iloc[0:1]], ignore_index=True, sort=False)

                                                final_df.iloc[index,7]= value
                                                index+=1
                                                if_else_run_track=1



                                if if_else_run_track==1:
                                    yesterday= datetime.strftime(datetime.now()-timedelta(1), "%d-%b-%y")  
                                    ubab=final_df.iloc[:,2:8]              ####### Added to drop any duplicates (checking values in column 2 to 8)
                                    
                                    if len(ubab)-len(ubab.drop_duplicates())!=0:
                                        index= index- (len(ubab)-len(ubab.drop_duplicates()))
                                    
                                    ubab.drop_duplicates(inplace=True)
                                    final_df=final_df.iloc[ubab.index,:]
                                    final_df.DATE= yesterday 

                                    ##### THE VALUES THAT HAS TO BE DELETED IN THE OVERSIMPLIFIED FILE######
                                    final_df.Awarded= np.nan
                                    final_df["MoU Target 21-22"]= np.nan
                                    final_df["Target"]=np.nan
                                    final_df["5 Years Targets"]=np.nan
    
                                    ########################################################################
                                    final_df= final_df.iloc[:,0:41]
    
    
    
        #                             final_df['DATE']='=today()-1'
        #                             final_df['PROGRESS Inch-Mtr']= '=IF(OR([@ACTIVITY]="Steel Laying", [@ACTIVITY]="PE Laying"), [@[PROGRESS Mtr]]*[@[Inch Multiplier]], 0)'
        #                             final_df['Pipe size unit']='=IF(RIGHT([@SIZE],1)="m","mm","""")'
        #                             final_df['Check1']='=LEN(K43836)'
        #                             final_df['Pipe size value']='=LEFT([@SIZE],(LEN([@SIZE])-L43836))'
        #                             final_df['Inch Multiplier']='=IF([@ACTIVITY]="PE Laying", [@[Pipe size value]]/25.4, [@[Pipe size value]])'
        #                             final_df['Check 3']='=$C$1-[@DATE]+1'
        #                             final_df['PROGRESS Mtr (Today)']='=IF([@[Check 3]]<2,[@[PROGRESS Mtr]],0)'
        #                             final_df["TODAY\'s PROGRESS INCH-M"]='=IF([@[Check 3]]<2, [@[PROGRESS Inch-Mtr]], 0)'
        #                             final_df['Today\'s Progress (All)']='=IF([@Project]= "STEEL +MDPE", [@[Total Inch-Km Progress Today]], [@[PROGRESS Mtr (Today)]])'
        #                             final_df['PROGRESS Mtr (Last 7 days)2']='=IF([@[Check 3]]<8,[@[PROGRESS Mtr]],0)'
        #                             final_df['PROGRESS Mtr (Last 7 days) INCH-KM']='=@IF([@[PROGRESS Mtr (Last 7 days)2]]=0,0,I:I)'
        #                             final_df['PROGRESS Mtr (last 15 days)']='=IF([@[Check 3]]<16,[@[PROGRESS Mtr]],0)'
        #                             final_df['PROGRESS Mtr (last 15 days) INCH-KM']='=@IF([@[PROGRESS Mtr (last 15 days)]]=0,0,I:I)'
        #                             final_df['PROGRESS Mtr (last 30 days)']='=IF([@[Check 3]]<31,[@[PROGRESS Mtr]],0)'
        #                             final_df['PROGRESS Mtr (last 30 days) INCH-KM']='=@IF([@[PROGRESS Mtr (last 30 days)]]=0,0,I:I)'
        #                             final_df['PROGRESS Mtr (last 90 days)']='=IF([@[Check 3]]<91,[@[PROGRESS Mtr]],0)'
        #                             final_df['PROGRESS Mtr (last 90 days) INCH-KM']='=@IF([@[PROGRESS Mtr (last 90 days)]]=0,0,I:I)'
        #                             final_df['Filter for Previous FY']='=IF([@DATE]<$AV$2, "Previous FY", "Current FY")'
    
    
    
    
                                    writer = pd.ExcelWriter(f"{os.getcwd()}\\OVERSIMPLIFIED_FILES\\{file_name.split('_')[0]}_nis_oversimplified.xlsx", engine='xlsxwriter')
                                    final_df.to_excel(writer,sheet_name= file_name.split('_')[0], index=False)
    
                                    (max_row, max_col) = final_df.shape
                                    workbook = writer.book
                                    worksheet = writer.sheets[file_name.split('_')[0]]
    
                                    worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})
                                    column_settings = [{'header': column} for column in final_df.columns]
                                    worksheet.set_column(0, max_col - 1, 12)                                       
    
    
    
                                    writer.save()
                                else:
                                    pass
                            if final_df.empty==False:
                                simplified_sum_for_check= df2.iloc[-1:].fillna(0).sum().sum() # to check the total sum for simplified and oversimplified
                                oversimplified_sum_for_check = final_df['PROGRESS Mtr'].sum()
                            
                                if abs(simplified_sum_for_check-oversimplified_sum_for_check) <=0.0001:
                                    print(f"{Fore.GREEN} {Style.BRIGHT} No mismatch in total for simplified and oversimplified file: {file_name.split('_')[0]} {Fore.RESET} {Style.RESET_ALL} ")
                                    print(f"simplified: {simplified_sum_for_check}, oversimplified: {oversimplified_sum_for_check}")
                                    mismatch_track+=1
                                else:
                                    print(f"{Fore.CYAN} {Style.BRIGHT} Warning: MATCH OVERSIMPLIFIED AND SIMPLIFIED FILE FOR DIFFERENCE{Fore.MAGENTA} (check contractor scope!){Fore.RESET} for GA: {file_name.split('_')[0]} {Fore.RESET} {Style.RESET_ALL}")
                                    print(f"{Fore.YELLOW} {Back.BLACK} {Style.NORMAL} simplified: {simplified_sum_for_check}, oversimplified: {oversimplified_sum_for_check} {Fore.RESET} {Back.RESET} {Style.RESET_ALL}")

#                                 final_combined_df= final_combined_df.append(final_df)
                                final_combined_df = pd.concat([final_combined_df, final_df], ignore_index=True, sort=False)

                                both_sheets_flag=0
                            else:
                                pass


    ################## THOSE THAT HAS "PE Laying" values in KM, change them here to M ##############
    if final_combined_df.empty==False:
        final_combined_df.loc[(final_combined_df["ACTIVITY"]=="PE Laying") & (final_combined_df["CITY"]=='Patna'),'PROGRESS Mtr']*= 1000
        final_combined_df.loc[(final_combined_df["ACTIVITY"]=="PE Laying") & (final_combined_df["CITY"]=='Jamshedpur'),'PROGRESS Mtr']*= 1000
        final_combined_df.loc[(final_combined_df["ACTIVITY"]=="Steel Laying") & (final_combined_df["CITY"]=='Jamshedpur'),'PROGRESS Mtr']*= 1000
        final_combined_df.loc[(final_combined_df["ACTIVITY"]=="Steel Laying") & (final_combined_df["CITY"]=='Patna'),'PROGRESS Mtr']*= 1000
        final_combined_df.loc[(final_combined_df["ACTIVITY"]=="Welding") & (final_combined_df["CITY"]=='Jamshedpur'),'PROGRESS Mtr']*= 1000
        final_combined_df.loc[(final_combined_df["ACTIVITY"]=="Welding") & (final_combined_df["CITY"]=='Patna'),'PROGRESS Mtr']*= 1000
        final_combined_df.loc[(final_combined_df["ACTIVITY"]=="PE Laying") & (final_combined_df["CITY"]=='Bhubaneshwar'),'PROGRESS Mtr']*= 1000
        final_combined_df.loc[(final_combined_df["ACTIVITY"]=="Welding") & (final_combined_df["CITY"]=='Bhubaneshwar'),'PROGRESS Mtr']*= 1000
        final_combined_df.loc[(final_combined_df["ACTIVITY"]=="Steel Laying") & (final_combined_df["CITY"]=='Bhubaneshwar'),'PROGRESS Mtr']*= 1000
        final_combined_df.loc[(final_combined_df["ACTIVITY"]=="PE Laying") & (final_combined_df["CITY"]=='Cuttack'),'PROGRESS Mtr']*= 1000
        final_combined_df.loc[(final_combined_df["ACTIVITY"]=="Welding") & (final_combined_df["CITY"]=='Cuttack'),'PROGRESS Mtr']*= 1000
        final_combined_df.loc[(final_combined_df["ACTIVITY"]=="Steel Laying") & (final_combined_df["CITY"]=='Cuttack'),'PROGRESS Mtr']*= 1000
        final_combined_df.loc[(final_combined_df["ACTIVITY"]=="Charged") & (final_combined_df["CITY"]=='Patna'),'PROGRESS Mtr']*= 1000
        final_combined_df.loc[(final_combined_df["ACTIVITY"]=="Charged") & (final_combined_df["CITY"]=='Jamshedpur'),'PROGRESS Mtr']*= 1000
        final_combined_df.loc[(final_combined_df["ACTIVITY"]=="Charged") & (final_combined_df["CITY"]=='Cuttack'),'PROGRESS Mtr']*= 1000
        final_combined_df.loc[(final_combined_df["ACTIVITY"]=="Charged") & (final_combined_df["CITY"]=='Bhubaneshwar'),'PROGRESS Mtr']*= 1000

    else:
        pass


    ###########################################################################################

    filename=f"{os.getcwd()}\\OVERSIMPLIFIED_FILES\\all_nis_oversimplified.xlsx"
    writer= pd.ExcelWriter(filename,engine= 'xlsxwriter')                   
    final_combined_df.dropna(subset=['PROGRESS Mtr'],inplace=True)
    final_combined_df= final_combined_df.iloc[:,0:41]
    final_combined_df.to_excel(writer,sheet_name= "ALL CITIES", index=False)
    (max_row, max_col) = final_combined_df.shape
    workbook = writer.book
    worksheet = writer.sheets["ALL CITIES"]
    worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})
    column_settings = [{'header': column} for column in final_combined_df.columns]
    worksheet.set_column(0, max_col - 1, 12)

    # filename_macro= filename.split('.')[0]+'.xlsm'
    # workbook= writer.book
    # workbook.filename= filename_macro
    # workbook.add_vba_project('vbaProject.bin')

    writer.save()
    writer.close()
else:
    print("Check the folder: Not all the files have been received / duplicate files received")
try:
    pickle_in.close()
    pickle_in_steel.close()
    pickle_in_mdpe.close()
except NameError:
    pass
if mismatch_track!=6:
    try:
        os.mkdir("C:\\Users\\hp\\Pictures\\failed_stage_backup")
    except FileExistsError:
        pass
    shutil.rmtree("C:\\Users\\hp\\Pictures\\failed_stage_backup")
    shutil.copytree("C:\\Users\\hp\\Documents\\Python Scripts","C:\\Users\\hp\\Pictures\\failed_stage_backup")

try:
    del xls
except NameError:
    pass

  0%|          | 0/26 [00:00<?, ?it/s]

  No mismatch in total for simplified and oversimplified file: Bhubaneshwar   
simplified: 78.417, oversimplified: 78.417
  No mismatch in total for simplified and oversimplified file: Cuttack   
simplified: 52.224, oversimplified: 52.224000000000004
  No mismatch in total for simplified and oversimplified file: Jamshedpur   
simplified: 184.214, oversimplified: 184.214
  No mismatch in total for simplified and oversimplified file: Patna   
simplified: 152.421, oversimplified: 152.421
  No mismatch in total for simplified and oversimplified file: Ranchi   
simplified: 630, oversimplified: 630
  No mismatch in total for simplified and oversimplified file: Varanasi   
simplified: 3384.3, oversimplified: 3384.3


### To restore the previous state forcefully, call the following function:

In [19]:
def reset_state():
    try:
        pickle_in.close()
        pickle_in_steel.close()
        pickle_in_mdpe.close()
        restore_state()
    except NameError:
        pickle_in.close()
        restore_state()

### Taking care of the Data Bank file

In [20]:
%%time
if mismatch_track==6:
    if (run_only_if_all_files_are_present==1) and (len(duplicacy_check) == 0):
        os.system('start excel.exe "C:\\Users\\hp\\Documents\\Python Scripts\\Varanasi_DPR.xlsm"')
        os.system('start excel.exe "C:\\Users\\hp\\Documents\\Python Scripts\\Bhubaneshwar_DPR.xlsm"')
        os.system('start excel.exe "C:\\Users\\hp\\Documents\\Python Scripts\\Patna_DPR.xlsm"')
        os.system('start excel.exe "C:\\Users\\hp\\Documents\\Python Scripts\\Ranchi_DPR.xlsm"')
        os.system('start excel.exe "C:\\Users\\hp\\Documents\\Python Scripts\\Jamshedpur_DPR.xlsm"')
        os.system('start excel.exe "C:\\Users\\hp\\Documents\\Python Scripts\\Cuttack_DPR.xlsm"')
        
        sleep(3)
        os.system('start chrome.exe')     
#         excel = win32com.client.Dispatch("Excel.Application")
        
#         hwnd = excel.Hwnd
        
#         win32gui.ShowWindow(hwnd, win32con.SW_MINIMIZE)

        
        xl= win32com.client.Dispatch('Excel.Application')
        xl.DisplayAlerts = False
        xl.Workbooks.Open(Filename="C:\\Users\\hp\\Documents\\Python Scripts\\DPR_MACRO.xlsm",ReadOnly=1)
        sleep(10)
        xl.Application.Run("DPR_MACRO.xlsm!Module1.Macro1")
        excel = win32com.client.GetObject(None, "Excel.Application")
        for window in excel.Windows:
            if window.Caption.startswith("Microsoft Excel"):
                window.Close()
                
        xl.Application.Quit()
        del xl
        os.system("""taskkill /f /im excel.exe""")
    else:
        print("Macros won't run, since all files have not been received.")
else:
    print("There seems to be some mismatch in the Oversimplified and Simplified file. Please check for mistakes.")

#### If any instance of excel is still open, I am using the below to kill all instances of excel:

def close_excel_instances():
    try:
        xl = win32com.client.GetObject(None, "Excel.Application")
        xl.DisplayAlerts = False
        xl.AlertBeforeOverwriting = False
        for process in psutil.process_iter():
            if process.name() == "EXCEL.EXE":
                process.kill()
    except:
        pass
close_excel_instances()

os.system("""taskkill /f /im chrome.exe""")

CPU times: total: 109 ms
Wall time: 2min 8s


0

### Sending file to "nishantsinha0502@gmail.com" for review:

In [21]:
##### For date in hindi #####
os.system("""taskkill /f /im excel.exe""")
yesterday=datetime.today()-timedelta(days=1)
hindi_number_list=[]
for i in range(2406,2416):
    hindi_number_list.append(chr(i))

english_number_list=[]
for i in range(48,58):
    english_number_list.append(chr(i))
    
months_in_hindi=["जनवरी","फरवरी"," मार्च ","अप्रैल","मई","जून","जुलाई","अगस्त","सितम्बर","अक्टूबर","नवम्बर","दिसम्बर"]
months_in_english=["January","February","March","April","May","June","July","August","September","October","November","December"]


part_1=[]
for i in yesterday.strftime("%d %B %Y").split(" ")[0]:
        part_1.append(hindi_number_list[int(i)])
part_1=''.join(part_1)

part_2= months_in_hindi[months_in_english.index(yesterday.strftime("%d %B %Y").split(" ")[1])]

part_3=[]
for i in yesterday.strftime("%d %B %Y").split(" ")[2]:
        part_3.append(hindi_number_list[int(i)])
part_3=''.join(part_3)

hindi_date= part_1+" "+part_2+" "+part_3

#####################

os.system("""taskkill /f /im excel.exe""")
if mismatch_track==6:
    if (run_only_if_all_files_are_present==1) and (len(duplicacy_check) == 0):
        sleep(8)
        try:
            os.rename("C:\\Users\\hp\\Documents\\Python Scripts\\Varanasi_DPR.xlsm",f"C:\\Users\\hp\\Documents\\Python Scripts\\{remember_me_1}")
        except (FileNotFoundError,PermissionError):
            pass
        try:
            os.rename("C:\\Users\\hp\\Documents\\Python Scripts\\Bhubaneshwar_DPR.xlsm",f"C:\\Users\\hp\\Documents\\Python Scripts\\{remember_me_2}")
        except (FileNotFoundError,PermissionError):    
            pass
        try:
            os.rename("C:\\Users\\hp\\Documents\\Python Scripts\\Patna_DPR.xlsm",f"C:\\Users\\hp\\Documents\\Python Scripts\\{remember_me_3}")
        except (FileNotFoundError,PermissionError):
            pass
        try:
            os.rename("C:\\Users\\hp\\Documents\\Python Scripts\\Ranchi_DPR.xlsm",f"C:\\Users\\hp\\Documents\\Python Scripts\\{remember_me_4}")
        except (FileNotFoundError,PermissionError):
            pass
        try:
            os.rename("C:\\Users\\hp\\Documents\\Python Scripts\\Jamshedpur_DPR.xlsm",f"C:\\Users\\hp\\Documents\\Python Scripts\\{remember_me_5}")
        except (FileNotFoundError,PermissionError):
            pass
        try:
            os.rename("C:\\Users\\hp\\Documents\\Python Scripts\\Cuttack_DPR.xlsm",f"C:\\Users\\hp\\Documents\\Python Scripts\\{remember_me_6}")
        except (FileNotFoundError,PermissionError):
            pass
        for process in psutil.process_iter():
            if "excel" in process.name().lower() :
                process.kill()
        yesterday=datetime.today()-timedelta(days=1)
        try:
            os.chdir(os.getcwd()+'\\COMBINED_DPR')
        except FileNotFoundError:
            os.chdir(os.getcwd())
            
        comb_dpr_files= os.listdir()

        msg= EmailMessage()
        sender= 'gailcgdwork@gmail.com'
        
        msg['From']=sender
        msg['To']=", ".join(recipients)
        msg['CC']=", ".join(cc)
        msg['Subject']= f"संयुक्त दैनिक प्रगति विवरण - दिनांक '{hindi_date}' तक की प्रगति / Combined DPR - Progress till {yesterday.strftime('%d-%b-%y')} | 6 Cities CGD Project"
        msg.set_content(

        f"""आदरणीय महोदय/Dear Sir,

कृपया छह शहरों में परियोजना संबंधी निर्माण स्थलों से प्राप्त दैनिक प्रगति विवरण के आधार पर दिनांक {hindi_date}  तक का संयुक्त दैनिक प्रगति विवरण प्राप्त करें। विभिन्न चरों की तुलना करने वाली पट्टी तालिका नीचे प्रस्तुत है:
Please find attached, the combined DPR with Progress till {yesterday.strftime("%d %B %Y")}, prepared, on the basis of site DPRs received. Bar charts comparing different variables are produced below: 



उपरोक्त दैनिक प्रगति विवरण का संदर्भ-चित्र नीचे प्रस्तुत है:
Snapshot of the above mentioned combined DPR is captured below for ready reference:



संयुक्त दैनिक प्रगति विवरण (दै.प्र.वि.) - संपीडित प्राकृतिक गैस स्टेशन:
Combined Daily Progress Report (DPR) - CNG stations:



यह अधोहस्ताक्षरी द्वारा स्वतः उत्पन्न विपत्र है:
THIS IS AN AUTOGENERATED EMAIL, ON BEHALF OF THE BELOW SIGNED:


सादर अभिवादन / Regards,
निशांत सिन्हा / Nishant Sinha


        """

        )

        for i in comb_dpr_files:
            try:
                with open(i, 'rb') as f:
                    file_data= f.read()
                    file_name= f.name
                    f.close()
            except PermissionError:
                continue
                
            if "Construction".lower() not in file_name.lower():
                if ".htm".lower() not in file_name.lower():
                    msg.add_attachment(file_data,maintype='application',subtype='octet-stream',filename=file_name)
                
        with smtplib.SMTP_SSL('smtp.gmail.com',465) as smtp:
            smtp.login('gailcgdwork@gmail.com', '<YOUR PASSWORD HERE>')
            smtp.send_message(msg)

#         %cd C:\Users\HM6176\hp\Documents\Python Scripts
            
            ## create a copy of the databank last 7 days ##
            
        timestr= strftime("%d%m%Y")
#         shutil.copyfile(f"C:\\Users\\hp\\Documents\\Python Scripts\\CGD_DPR_DATA_BANK.xlsm",f"C:\\Users\\hp\\Pictures\\databank_backup_last_7_days\\CGD_DPR_DATA_BANK_{timestr}.xlsm")
        shutil.make_archive(f"C:\\Users\\hp\\Pictures\\databank_backup_last_7_days\\DPR_BACKUP_STATE_{timestr}","zip",f"C:\\Users\\hp\\Documents\\Python Scripts\\")
        
        %cd C:\Users\HM6176\hp\Documents\Python Scripts\COMBINED_DPR
        
        os.startfile(f"{os.getcwd()}\\{[i for i in comb_dpr_files if '.xlsx' in i][0]} ")
        os.startfile(f"{os.getcwd()}\\{[i for i in comb_dpr_files if '.htm' in i][0]} ")
        
        for i in os.listdir("C:\\Users\\hp\\Pictures\\databank_backup_last_7_days"):
            if time()-os.path.getmtime(f"C:\\Users\\hp\\Pictures\\databank_backup_last_7_days\\{i}") > 6*24*60*60:
                os.remove(f"C:\\Users\\hp\\Pictures\\databank_backup_last_7_days\\{i}")
        
    else:
        print('No mail shall be sent. Not all files have been received.')
else:
    print("There seems to be some mismatch in the Oversimplified and Simplified file. Please check for mistakes.")

[WinError 3] The system cannot find the path specified: 'C:\\Users\\HM6176\\hp\\Documents\\Python Scripts\\COMBINED_DPR'
C:\Users\hp\Documents\Python Scripts


#### Sending files for reports every 15th and last day of month

In [22]:
if mismatch_track==6:
    if (run_only_if_all_files_are_present==1) and (len(duplicacy_check) == 0):
        os.chdir("C:\\Users\\hp\\Documents\\Python Scripts")
        if datetime.today().date().day in [16,1]:
            files_for_backup= ['BHUBANESHWAR','CUTTACK','JAMSHEDPUR','PATNA','RANCHI','VARANASI','CGD_DPR_DATA_BANK']
            msg= EmailMessage()
            sender= 'gailcgdwork@gmail.com'
            msg['From']=sender
            msg['To']= ", ".join(send_for_reporting)
            msg['Subject']= f'Autogenerated mail: Data Bank updated on: {datetime.now().strftime("%d/%m/%y %H:%M:%S")} with site DPRs'
            for i in os.listdir():
                for j in files_for_backup:
                    if j.lower() in i.lower():
                        if '.xlsm' in i or '.xlsb' in i:
                            with open(i, 'rb') as f:
                                file_data= f.read()
                                file_name= f.name 
                                f.close()
                            msg.add_attachment(file_data,maintype='application',subtype='octet-stream',filename=file_name)

            with smtplib.SMTP_SSL('smtp.gmail.com',465) as smtp:
                smtp.login('gailcgdwork@gmail.com', '<YOUR PASSWORD HERE>')
                smtp.send_message(msg)
            os.chdir('C:\\Users\\hp\\Documents')
            print("Message sent successfully")
 
        else:
            pass
    else:
        pass

In [23]:
os.chdir("C:\\Users\\hp\\Documents\\Python Scripts")
try:
    
    for i in os.listdir():
        if '.xlsx' in i:
            os.remove(i)
    
except FileNotFoundError:
    pass

In [24]:
if mismatch_track==6:
    if (run_only_if_all_files_are_present==1) and (len(duplicacy_check) == 0):
        root=Tk()
        tkinter.messagebox.showinfo('Do Not Forget to: ','Send the Construction DPR on WhatsApp')
        subprocess.Popen(["cmd", "/C", "start whatsapp://send?phone=+919990046918"], shell=True)
#         root.mainloop()
        root.destroy()

# New checks applied

In [25]:
if (mismatch_track==6 and (run_only_if_all_files_are_present==1 and (len(duplicacy_check) == 0))):
    df_main = pd.read_excel("CGD_DPR_DATA_BANK.xlsb",engine='pyxlsb',sheet_name="Data")
    df_main.columns=df_main.iloc[0]
    df_main.drop(index=0,inplace=True)
else:
    print("Not all files have been received.")

### 1 of 4 ---> checks for steel welding, steel laying and steel charging

In [26]:
if (mismatch_track==6 and (run_only_if_all_files_are_present==1 and (len(duplicacy_check) == 0))):

    check_flag_1= 0
    warning_in_part_1=0
    for i in df_main['CITY'].unique():
        st_lay_sum=df_main[(df_main["ACTIVITY"]=='Steel Laying') & (df_main["CITY"]==i) ]['PROGRESS Mtr'].sum()
        st_wel_sum=df_main[(df_main["ACTIVITY"]=='Welding') & (df_main["CITY"]==i) ]['PROGRESS Mtr'].sum()
        st_char_sum=df_main[(df_main["ACTIVITY"]=='Charged') & (df_main["CITY"]==i) & (df_main["Project"]=="Steel Network")]['PROGRESS Mtr'].sum()

        if st_lay_sum>st_wel_sum:
            print(f"{Fore.YELLOW} {Style.BRIGHT} Please Note: Steel Laying is {round(st_lay_sum-st_wel_sum,2)} M more than Steel Welding for {i} {Fore.RESET} {Style.RESET_ALL}")            
        else:
            print(f"{Fore.GREEN} {Style.BRIGHT} No issue found in Steel Laying data and Welding data for {i} {Fore.RESET} {Style.RESET_ALL}")
            check_flag_1+=1
        if st_char_sum>st_lay_sum:
            print(f"{Fore.YELLOW} {Style.BRIGHT} Please Note: Steel Charging is {round(st_char_sum-st_lay_sum,2)} M more than Steel Laying for {i} {Fore.RESET} {Style.RESET_ALL}")
        else:
            print(f"{Fore.GREEN} {Style.BRIGHT} No issue found in Steel Laying data and Charging data for {i} {Fore.RESET} {Style.RESET_ALL}")
            check_flag_1+=1
        print("\n ******************************************************************")
    if check_flag_1 < 12:
        warning_in_part_1=1
else:
    print("Not all files have been received")

  No issue found in Steel Laying data and Welding data for Varanasi  
  No issue found in Steel Laying data and Charging data for Varanasi  

 ******************************************************************
  No issue found in Steel Laying data and Welding data for Bhubaneshwar  
  No issue found in Steel Laying data and Charging data for Bhubaneshwar  

 ******************************************************************
  No issue found in Steel Laying data and Welding data for Ranchi  
  No issue found in Steel Laying data and Charging data for Ranchi  

 ******************************************************************
  No issue found in Steel Laying data and Welding data for Jamshedpur  
  No issue found in Steel Laying data and Charging data for Jamshedpur  

 ******************************************************************
  No issue found in Steel Laying data and Welding data for Cuttack  
  No issue found in Steel Laying data and Charging data for Cuttack  

 ***********

### 2 of 4 ---> For MDPE Laying vs Charging

In [27]:
if (mismatch_track==6 and (run_only_if_all_files_are_present==1 and (len(duplicacy_check) == 0))):

    check_flag_2=0
    warning_in_part_2=0
    for i in df_main['CITY'].unique():
        pe_lay_sum=df_main[(df_main["ACTIVITY"]=='PE Laying') & (df_main["CITY"]==i) ]['PROGRESS Mtr'].sum()
        pe_char_sum=df_main[(df_main["ACTIVITY"]=='Charged') & (df_main["CITY"]==i) & (df_main["Project"]=='MDPE Network')]['PROGRESS Mtr'].sum()    
        if st_char_sum>st_lay_sum:
            print(f"{Fore.YELLOW} {Style.BRIGHT} Please Note: MDPE Charging is {round(pe_char_sum-pe_lay_sum,2)} M more than Steel Laying for {i} {Fore.RESET} {Style.RESET_ALL}")
        else:
            print(f"{Fore.GREEN} {Style.BRIGHT} No issue found in MDPE Laying data and Charging data for {i} {Fore.RESET} {Style.RESET_ALL}")
            check_flag_2+=1

    if check_flag_2 < 6:
        warning_in_part_2=1

else:
    print("Not all files have been received")

  No issue found in MDPE Laying data and Charging data for Varanasi  
  No issue found in MDPE Laying data and Charging data for Bhubaneshwar  
  No issue found in MDPE Laying data and Charging data for Ranchi  
  No issue found in MDPE Laying data and Charging data for Jamshedpur  
  No issue found in MDPE Laying data and Charging data for Cuttack  
  No issue found in MDPE Laying data and Charging data for Patna  


### 3 of 4 ---> For LMC Riser, Meter and Conversion

In [28]:
if (mismatch_track==6 and (run_only_if_all_files_are_present==1 and (len(duplicacy_check) == 0))):



    check_flag_3=0
    warning_in_part_3=0

    for i in df_main['CITY'].unique():
        riser_sum=df_main[(df_main["ACTIVITY"]=='Connection with Riser') & (df_main["CITY"]==i) ]['PROGRESS Mtr'].sum()
        meter_sum=df_main[(df_main["ACTIVITY"]=='Connection with Meter') & (df_main["CITY"]==i) ]['PROGRESS Mtr'].sum() 
        conversion_sum=df_main[(df_main["ACTIVITY"]=='PNG Conversion') & (df_main["CITY"]==i) ]['PROGRESS Mtr'].sum() 

        if riser_sum < meter_sum:
            print(f"{Fore.YELLOW} {Style.BRIGHT} Please Note: Number of Meter is {round(meter_sum-riser_sum,2)} more for {i} {Fore.RESET} {Style.RESET_ALL}")
        else:
            print(f"{Fore.GREEN} {Style.BRIGHT} No issue found in Riser and Meter data for {i} {Fore.RESET} {Style.RESET_ALL}")
            check_flag_3+=1
        if meter_sum < conversion_sum:
            print(f"{Fore.YELLOW} {Style.BRIGHT} Please Note: Number of Conversion is {round(conversion_sum-meter_sum,2)} more for {i} {Fore.RESET} {Style.RESET_ALL}")
        else:
            print(f"{Fore.GREEN} {Style.BRIGHT} No issue found in Meter and Conversion data for {i} {Fore.RESET} {Style.RESET_ALL}")
            check_flag_3+=1
        if riser_sum < conversion_sum:
            print(f"{Fore.YELLOW} {Style.BRIGHT} Please Note: Number of Meter is {round(conversion_sum-riser_sum,2)} more for {i} {Fore.RESET} {Style.RESET_ALL}")
        else:
            print(f"{Fore.GREEN} {Style.BRIGHT} No issue found in Riser and Conversion data for {i} {Fore.RESET} {Style.RESET_ALL}")
            check_flag_3+=1

        print("\n************************************************")
    if check_flag_3 < 18:
        warning_in_part_3=1   
        
else:
    print("Not all files have been received")

  No issue found in Riser and Meter data for Varanasi  
  No issue found in Meter and Conversion data for Varanasi  
  No issue found in Riser and Conversion data for Varanasi  

************************************************
  No issue found in Riser and Meter data for Bhubaneshwar  
  No issue found in Meter and Conversion data for Bhubaneshwar  
  No issue found in Riser and Conversion data for Bhubaneshwar  

************************************************
  No issue found in Riser and Meter data for Ranchi  
  No issue found in Meter and Conversion data for Ranchi  
  No issue found in Riser and Conversion data for Ranchi  

************************************************
  No issue found in Riser and Meter data for Jamshedpur  
  No issue found in Meter and Conversion data for Jamshedpur  
  No issue found in Riser and Conversion data for Jamshedpur  

************************************************
  No issue found in Riser and Meter data for Cuttack  
  No issue found in M

## 4 of 4 ---> Abnormal Value Check

In [29]:
if (mismatch_track==6 and (run_only_if_all_files_are_present==1 and (len(duplicacy_check) == 0))):


    check_flag_4=0
    warning_in_part_4=0

    aaj_ke_dpr_ka_progress=df_main[df_main["DATE"]== df_main["DATE"].tail(10000).unique().max()]
    riser_flag=meter_flag=conversion_flag=200
    pe_lay_flag= 1500
    welding_flag= 600
    steel_lay_flag= 600
    for i in df_main['CITY'].unique():
        print("\n")
        print(f"*****************  For {i}: ----------->")
        riser_sum_today=aaj_ke_dpr_ka_progress[(aaj_ke_dpr_ka_progress["ACTIVITY"]=='Connection with Riser') & (df_main["CITY"]==i) ]['PROGRESS Mtr'].sum()
        meter_sum_today=aaj_ke_dpr_ka_progress[(aaj_ke_dpr_ka_progress["ACTIVITY"]=='Connection with Meter') & (df_main["CITY"]==i) ]['PROGRESS Mtr'].sum()
        pe_lay_sum_today=aaj_ke_dpr_ka_progress[(aaj_ke_dpr_ka_progress["ACTIVITY"]=='PE Laying') & (df_main["CITY"]==i) ]['PROGRESS Mtr'].sum() 
        png_conv_today= aaj_ke_dpr_ka_progress[(aaj_ke_dpr_ka_progress["ACTIVITY"]=='PNG Conversion') & (df_main["CITY"]==i) ]['PROGRESS Mtr'].sum()
        welding_today= aaj_ke_dpr_ka_progress[(aaj_ke_dpr_ka_progress["ACTIVITY"]=='Welding') & (df_main["CITY"]==i) ]['PROGRESS Mtr'].sum()
        steel_laying_today= aaj_ke_dpr_ka_progress[(aaj_ke_dpr_ka_progress["ACTIVITY"]=='Steel Laying') & (df_main["CITY"]==i) ]['PROGRESS Mtr'].sum()

        if riser_sum_today > riser_flag:
            print(f"{Fore.YELLOW} {Style.BRIGHT} Abnormally high value ({riser_sum_today}) for Riser in {i} {Fore.RESET} {Style.RESET_ALL}")
        else:
            print(f"{Fore.GREEN} {Style.BRIGHT}Reported riser figure is below {riser_flag} for {i} {Fore.RESET} {Style.RESET_ALL}")
            check_flag_4+=1

        if meter_sum_today > meter_flag:
            print(f"{Fore.YELLOW} {Style.BRIGHT} Abnormally high value ({meter_sum_today}) for Meter in {i} {Fore.RESET} {Style.RESET_ALL}")
        else:
            print(f"{Fore.GREEN} {Style.BRIGHT}Reported meter figure is below {meter_flag} for {i} {Fore.RESET} {Style.RESET_ALL}")
            check_flag_4+=1


        if png_conv_today > conversion_flag:
            print(f"{Fore.YELLOW} {Style.BRIGHT} Abnormally high value ({png_conv_today}) for Domestic Conversion in {i} {Fore.RESET} {Style.RESET_ALL}")
        else:
            print(f"{Fore.GREEN} {Style.BRIGHT}Reported conversion figure is below {conversion_flag} for {i} {Fore.RESET} {Style.RESET_ALL}")
            check_flag_4+=1


        if pe_lay_sum_today > pe_lay_flag:
            print(f"{Fore.YELLOW} {Style.BRIGHT} Abnormally high value ({pe_lay_sum_today}) M for PE Laying in {i} {Fore.RESET} {Style.RESET_ALL}")
        else:
            print(f"{Fore.GREEN} {Style.BRIGHT}Reported PE laying figure is below {pe_lay_flag} M for {i} {Fore.RESET} {Style.RESET_ALL}")
            check_flag_4+=1


        if welding_today > welding_flag:
            print(f"{Fore.YELLOW} {Style.BRIGHT} Abnormally high value for Welding ({welding_today}) M in {i} {Fore.RESET} {Style.RESET_ALL}")
        else:
            print(f"{Fore.GREEN} {Style.BRIGHT}Reported welding figure is below {welding_flag} M for {i} {Fore.RESET} {Style.RESET_ALL}")
            check_flag_4+=1

        if steel_laying_today > steel_lay_flag:
            print(f"{Fore.YELLOW} {Style.BRIGHT} Abnormally high value for Steel Laying ({steel_laying_today}) in {i} {Fore.RESET} {Style.RESET_ALL}")
        else:
            print(f"{Fore.GREEN} {Style.BRIGHT}Reported steel laying figure is below {steel_lay_flag} M for {i} {Fore.RESET} {Style.RESET_ALL}")
            check_flag_4+=1

    if check_flag_4 < 36:
        warning_in_part_4=1     
        
else:
    print("Not all files have been received")



*****************  For Varanasi: ----------->
 Reported riser figure is below 200 for Varanasi  
 Reported meter figure is below 200 for Varanasi  
 Reported conversion figure is below 200 for Varanasi  
 Reported PE laying figure is below 1500 M for Varanasi  
 Reported welding figure is below 600 M for Varanasi  
 Reported steel laying figure is below 600 M for Varanasi  


*****************  For Bhubaneshwar: ----------->
 Reported riser figure is below 200 for Bhubaneshwar  
 Reported meter figure is below 200 for Bhubaneshwar  
 Reported conversion figure is below 200 for Bhubaneshwar  
 Reported PE laying figure is below 1500 M for Bhubaneshwar  
 Reported welding figure is below 600 M for Bhubaneshwar  
 Reported steel laying figure is below 600 M for Bhubaneshwar  


*****************  For Ranchi: ----------->
 Reported riser figure is below 200 for Ranchi  
 Reported meter figure is below 200 for Ranchi  
 Reported conversion figure is below 200 for Ranchi  
 Reported PE lay

### FOR SPEECH INITIATION

In [30]:
def warning_speech(part):
    engine = pyttsx3.init()

    text = f"Have a look, it seems there is a warning in part {part}"
    engine.setProperty('voice', 'english_rp+f3')
    engine.setProperty('rate',160)
    engine.say(text)

    engine.runAndWait()

In [31]:
if (mismatch_track==6 and (run_only_if_all_files_are_present==1 and (len(duplicacy_check) == 0))):


    def show_warning(part):
        root = Tk()
        tkinter.messagebox.showinfo(title=f"Warning Message (Part {part})", message=f"There is a warning in Part {part}. Please check")
        warning_speech(part)
        root.destroy()

    warnings = 0
    for part in range(1, 5):
        if locals()[f"warning_in_part_{part}"]:
            show_warning(part)
            warnings += 1

    if warnings == 0:
        root = Tk()
        tkinter.messagebox.showinfo(title="No Warning :) ", message="There     is     no     Warning. \n\n Mail Aage Bhej Do Nishant.")
        root.destroy()
        
else:
    print("Not all files have been received")

## General Notes:

In [32]:
print(f"Total Number of days: {dpr_delay_count.day.count()} since first implementation of tracking on 24th July 2022: \n")
print("GA wise delay count is as below: \n")
print(dpr_delay_count.iloc[:,1:].sum())

Total Number of days: 525 since first implementation of tracking on 24th July 2022: 

GA wise delay count is as below: 

varanasi        70
bhubaneshwar    21
patna           90
ranchi          83
jamshedpur      16
cuttack         58
dtype: object


In [33]:
print("Reminder count for the day is as below:\n ")
print(dpr_reminder_count)

Reminder count for the day is as below:
 
   varanasi  bhubaneshwar  patna  ranchi  jamshedpur  cuttack
0         0             0      0       0           0        0


कल की प्रगति दर्शाने वाली दैनिक प्रगति विवरण इस विपत्र के प्रेषण तक रांची से प्राप्त नहीं हुई है। अतः संयुक्त दैनिक प्रगति प्रतिवेदन में रांची की प्रगति को शून्य के रूप में अद्यतन किया गया है।

Daily progress report, showing yesterday's progress has not been received from Ranchi till the time of this email. Hence, the progress for Ranchi has been updated as zero in combined daily progress report.

## Code generation for Macros [SITE DPR]

### For addition of new lines, to clearing the "today" content in the Steel Sheet, run the below code and enter the column:


In [34]:
# a= """Sheets("01.Steel Laying").Range("""
# b= '"F10:F13,F15:F18,F20:F23,F25:F28,F30:F33,F35:F38,F39:F45"'
# c= ").ClearContents"
# old_col="F"
# new_column= 1
# while new_column!= "n":
#     new_col_name= input("Enter the Column name: ")
#     b= b.replace(old_col,new_col_name)
#     old_col=new_col_name
#     print("Paste the code below in macros: ")
#     print('\n')
#     print(a+b+c)
#     print('\n')
#     new_column=input("another column? Press 'n' to exit, enter to continue: ")
    

**Note: for the MDPE part, the code is simple and may be copied from Macros**

### For addition of lines, to update the previous value in the Steel Sheet, run the below code and enter the columns. Copy the output and paste in the macros:

In [35]:
# string1= """Sheets("01.Steel Laying").Range("""
# list2=[('"G10: G13"'),('"E10: E13"'),('"G15: G18"'),('"E15: E18"'),('"G20: G23"'),('"E20: E23"'),
#        ('"G25: G28"'),('"E25: E28"'),('"G30: G33"'),('"E30: E33"'),('"G35: G38"'),('"E35: E38"'),('"G39: G45"'),
#        ('"E39: E45"')]
# string3=").Copy"
# string4=").PasteSpecial xlPasteValues"

# new_column= 1
# while new_column!= "n":

#     cumulative_column= input("Enter the Cumulative Column: ")
#     previous_column = input("Enter the Previous Column: ")
#     print("Copy and paste the code below: \n")

#     for i,j in enumerate(list2):
#         if(int(i)%2==0):
#             print(string1+j.replace("G",cumulative_column)+string3)
#         elif(int(i)%2==1):
#             print(string1+j.replace("E",previous_column)+string4)
#     print('\n')
#     new_column=input("another column? Press 'n' to exit, enter to continue: ")
#     print('\n')

**Note: for the MDPE part, the code is simple and may be copied from Macros**

_Additional Notes: Uncomment the above two cells and run, in case of addition of new vendor_